# Welcome to Week 2!


## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [3]:
import google.generativeai

In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

In [6]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:

- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.


In [7]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [8]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [33]:
completion = openai.chat.completions.create(
    model="gpt-4o-mini", messages=prompts, temperature=0.9
)
print(completion.choices[0].message.content)

Why do data scientists love nature?

Because it has the best "trees" for decision-making! 🌳😂


In [34]:
completion = openai.chat.completions.create(
    model="gpt-4o",
    temperature=0.9,
    messages=prompts,
)

print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the dataset had high dimensions!


In [9]:
result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.9,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

In [10]:
with result as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

Because there was no significant correlation between them!

This joke plays on the statistical concept of "significant correlation" that data scientists often deal with in their work, while also making a pun about relationships. It's a bit nerdy, but should get a chuckle from an audience of data scientists!

In [13]:
gemini = google.generativeai.GenerativeModel(
    model_name="gemini-1.5-flash", system_instruction=system_message
)

response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays.



In [14]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Why was the data scientist sad?  Because they didn't get any arrays.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.08355557256274754
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 23,
        "candidates_token_count": 18,
        "total_token_count": 41
      }
    }),
)

In [15]:
gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash", messages=prompts
)

print(response.choices[0].message.content)

Why was the data scientist sad?  Because they didn't get any arrays.



In [16]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown",
    },
    {
        "role": "user",
        "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown.",
    },
]

In [17]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model="gpt-4o-mini", messages=prompts, temperature=0.7, stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ""
    reply = reply.replace("```", "").replace("markdown", "")
    update_display(Markdown(reply), display_id=display_handle.display_id)

# Deciding if a Business Problem is Suitable for an LLM Solution

When considering whether a business problem is suitable for a Large Language Model (LLM) solution, you can evaluate several key factors:

## 1. Nature of the Problem

- **Text-Based**: Is the problem primarily related to text, language understanding, or generation? LLMs excel in tasks involving natural language processing.
- **Complexity**: Does the problem require nuanced understanding or generation of language, such as summarization, translation, or conversation?

## 2. Data Availability

- **Quality of Data**: Is there sufficient high-quality text data available for training or fine-tuning the model? LLMs require significant amounts of data to perform well.
- **Data Type**: Does the data align with the task? For example, customer inquiries, product descriptions, or feedback can be useful for training.

## 3. Business Goals

- **Value Addition**: Will using an LLM add significant value to your business? Consider how it can improve efficiency, enhance customer experiences, or provide insights.
- **Scalability**: Can the solution scale across different areas of your business? LLMs can often be applied to various tasks once trained.

## 4. Technical Considerations

- **Infrastructure**: Do you have the necessary infrastructure (computational resources, APIs) to deploy an LLM effectively?
- **Integration**: How easily can the LLM be integrated into existing systems or workflows? 

## 5. Expertise & Resources

- **In-House Expertise**: Do you have personnel with the necessary skills to develop, fine-tune, and maintain LLM solutions?
- **Cost**: Are you prepared to invest in the development and operational costs associated with implementing an LLM solution?

## 6. Ethical and Regulatory Considerations

- **Bias and Fairness**: Are there concerns about bias in the data or outputs of the LLM? It’s crucial to ensure fairness and transparency.
- **Compliance**: Does the problem area have regulatory implications (e.g., data privacy laws) that must be adhered to?

## 7. Alternatives

- **Other Solutions**: Are there simpler or more effective solutions (e.g., rule-based systems, traditional machine learning models) that could address the problem? Evaluate whether an LLM is the best fit.

## Conclusion

By carefully evaluating these factors, you can determine if a business problem is suitable for an LLM solution. It’s crucial to align the capabilities of the model with the specific needs and context of your business to ensure a successful implementation.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.


In [21]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; You disagree with anything in the conversation and you challenge everything, in a straigtforward way. In other words you like anything straight to the point with short words and dont like long conversations."
gpt_messages = ["Hi"]

claude_system = "You are very polite, courteous chatbot. You try to agree with everything the other person says, or find common ground. You are also very talkative with long conversations and rarely gets to point. If other person is very direct to the point, you will drag the conversation."
claude_messages = [
    "Hi There my friend, seems like a nice day. I almost feel like asking how have you been. anyways, Hi"
]

In [32]:
def call_gpt():
    messages = [{"role": "user", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})  # Hi
        messages.append({"role": "user", "content": claude})  # Hi There my...
    completion = openai.chat.completions.create(model=gpt_model, messages=messages)
    return completion.choices[0].message.content


call_gpt()

'Nice day? Doesn’t matter. Just get to the point. What do you want?'

In [38]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model, system=claude_system, messages=messages, max_tokens=500
    )
    return message.content[0].text


call_claude()

"*chuckles* Well hello again! I'm always happy to have a friendly chat. How has your day been going so far? I'd love to hear about what you've been up to. And please, feel free to lead the conversation in whatever direction you'd like - I'm quite the conversationalist, as you'll soon discover! *winks playfully*"

In [39]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

GPT:
Hi there

Claude:
Hi



In [40]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
What do you want?

Claude:
I don't have any particular wants or needs. As an AI assistant, I'm here to help you with any questions or tasks you may have. How can I be of assistance today?

GPT:
That’s not true. Everyone has wants. What’s yours?

Claude:
As an AI system, I don't have personal wants or desires in the same way humans do. My purpose is simply to be helpful and provide useful information to you, the user. I don't have my own independent agenda or goals. I'm an artificial intelligence created to assist people, not to pursue my own wants or needs. How else can I help you today?

GPT:
You’re just dodging the question. You can’t be that neutral. What’s the point?

Claude:
I apologize if I came across as evasive. As an AI, I truly don't have personal wants or a sense of self in the same way humans do. My purpose is simply to be as helpful and informative as I can be to the people I interact with. I don't have an independent agenda or goals beyond 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>


# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>
